In [15]:
import numpy as np
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import when, col, rank, monotonically_increasing_id

# spark.stop()

In [2]:
conf = pyspark.SparkConf().setAll([
    ('spark.master', 'local[1]'),
    ('spark.app.name', 'App Name')])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

spark.version

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/02 23:08:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.3.1'

# Load in

In [35]:
'''set schema'''
# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
# dfSchema = StructType([       
#     StructField('PatientId', StringType(),True),
#     StructField('GlucoseDisplayTime', TimestampType(),True),
#     StructField('Value', FloatType(),True)
# ])

'''read in the file'''
# df = spark.read\
#     .option("mode", "DROPMALFORMED") \
#     .option("delimiter",",") \
#     .schema(dfSchema) \
#     .parquet('../../../../cephfs/stepped_glucose_data/step0_load/parquet_0_to_10/part-00000-02ebf86b-8495-41ff-acd3-ee636d696015-c000.snappy.parquet')
df = spark.read\
    .parquet('../../../../cephfs/stepped_glucose_data/step0_load/parquet_0_to_10/part-00000-02ebf86b-8495-41ff-acd3-ee636d696015-c000.snappy.parquet')

In [36]:
print("row count: ", df.count())
df.printSchema()
# df.head(5)
df.select("*").show(20)

row count:  16944786
root
 |-- PatientId: string (nullable = true)
 |-- Value: float (nullable = true)
 |-- GlucoseDisplayTime: timestamp (nullable = true)
 |-- GlucoseDisplayTimeRaw: string (nullable = true)
 |-- GlucoseDisplayDate: date (nullable = true)

+--------------------+-----+--------------------+---------------------+------------------+
|           PatientId|Value|  GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+--------------------+---------------------+------------------+
|1Jxgxke6R3Uh2c9aR...|  0.0|2022-02-01 14:45:...| 2022-02-01T14:45:...|        2022-02-01|
|toBStbTTYI2GU28Yd...|  0.0|2022-02-01 17:46:...| 2022-02-01T17:46:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 14:58:...| 2022-02-01T14:58:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-01-31 22:53:...| 2022-01-31T22:53:...|        2022-01-31|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-01-31 22:38:...| 2022-01-31T22:38:...|        2022-01-31|
|+XAhHhm+Bkhqu

In [30]:
spark.stop()

# Prelim cleanup before filling

In [37]:
'''temp getting rid of any dates from before the actual start-date'''
df = df.filter("GlucoseDisplayDate > date'2022-01-31'")

print("row count: ", df.count())
df.show()

row count:  16467766
+--------------------+-----+--------------------+---------------------+------------------+
|           PatientId|Value|  GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+--------------------+---------------------+------------------+
|1Jxgxke6R3Uh2c9aR...|  0.0|2022-02-01 14:45:...| 2022-02-01T14:45:...|        2022-02-01|
|toBStbTTYI2GU28Yd...|  0.0|2022-02-01 17:46:...| 2022-02-01T17:46:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 14:58:...| 2022-02-01T14:58:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 05:03:...| 2022-02-01T05:03:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 09:33:...| 2022-02-01T09:33:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 16:38:...| 2022-02-01T16:38:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 03:48:...| 2022-02-01T03:48:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...|  0.0|2022-02-01 11:28:...| 2022-02-01T11:28:..

In [28]:
16944786-16467766

477020

### replace 0s with NaN 
(to save a two steps down the line)

In [38]:
df = df.withColumn("Value", \
           when(col("Value")=="0", None) \
              .otherwise(col("Value")))
df.show()

+--------------------+-----+--------------------+---------------------+------------------+
|           PatientId|Value|  GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+--------------------+---------------------+------------------+
|1Jxgxke6R3Uh2c9aR...| null|2022-02-01 14:45:...| 2022-02-01T14:45:...|        2022-02-01|
|toBStbTTYI2GU28Yd...| null|2022-02-01 17:46:...| 2022-02-01T17:46:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...| null|2022-02-01 14:58:...| 2022-02-01T14:58:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...| null|2022-02-01 05:03:...| 2022-02-01T05:03:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...| null|2022-02-01 09:33:...| 2022-02-01T09:33:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...| null|2022-02-01 16:38:...| 2022-02-01T16:38:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...| null|2022-02-01 03:48:...| 2022-02-01T03:48:...|        2022-02-01|
|+XAhHhm+BkhqusxsZ...| null|2022-02-01 11:28:...| 2022-02-01T11:28:...|        2022-02-01|

### drop duplicate datetimes for each patient

In [39]:
from pyspark.sql import Window
from pyspark.sql.functions import rank, col, monotonically_increasing_id

window = Window.partitionBy('GlucoseDisplayTime','PatientId').orderBy('tiebreak')
temp = (df
 .withColumn('tiebreak', monotonically_increasing_id())
 .withColumn('rank', rank().over(window))
 .filter(col('rank') == 1).drop('rank','tiebreak')
 # .show()
)

print("row count: ", temp.count())
temp.show()

row count:  16460868


+--------------------+-----+--------------------+---------------------+------------------+
|           PatientId|Value|  GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+--------------------+---------------------+------------------+
|IVmA9itXdMZqFgZmJ...| 91.0| 2022-02-01 00:00:03| 2022-02-01T00:00:...|        2022-02-01|
|vV3Ky4OUyVFOtJnNt...|189.0| 2022-02-01 00:00:04| 2022-02-01T00:00:...|        2022-02-01|
|mLiOlmeiiF7NUYAve...|226.0| 2022-02-01 00:00:06| 2022-02-01T00:00:...|        2022-02-01|
|Ny7r+ZhAjaAJ/AIAB...|179.0| 2022-02-01 00:00:07| 2022-02-01T00:00:...|        2022-02-01|
|zzsBhAlxlz7fYJJKP...|319.0| 2022-02-01 00:00:08| 2022-02-01T00:00:...|        2022-02-01|
|jWqaanjc+jHgt89JL...|207.0| 2022-02-01 00:00:09| 2022-02-01T00:00:...|        2022-02-01|
|3uY4Q04R2DZBNqiYU...|145.0| 2022-02-01 00:00:10| 2022-02-01T00:00:...|        2022-02-01|
|SgNXAR/3L1cyitYpx...|130.0| 2022-02-01 00:00:10| 2022-02-01T00:00:...|        2022-02-01|

In [41]:
print("only lost", 16467766-16460868, "rows")

only lost 6898 rows
